Install Dependencies

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import cv2
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

Load Dataset

In [ ]:
# Load the Dataset
def load_dataset(txt_file):
    data, labels = [], []
    with open(txt_file, 'r') as file:
        for line in file.readlines():
            image_path = line.strip()
            if os.path.exists(image_path):
                label = image_path.split('/')[1] 
                data.append(image_path)
                labels.append(label)
    return pd.DataFrame({'image_path': data, 'label': labels})

Define CNN Model

In [ ]:
# Simple CNN Model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc = nn.Linear(32*16*16, num_classes)
        
    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

Preprocess Data

In [ ]:
# Preprocessing
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (64, 64))
    image = image.transpose(2, 0, 1)
    return torch.FloatTensor(image) / 255.0

Results

In [ ]:
# Main Workflow - Training & Evaluation

def run_CNN(lr, epochs):
    train_df = load_dataset("train.txt")
    test_df = load_dataset("val.txt")
    
    X_train = torch.stack([preprocess_image(path) for path in train_df['image_path']])
    X_test = torch.stack([preprocess_image(path) for path in test_df['image_path']])
    
    encoder = LabelEncoder()
    y_train = encoder.fit_transform(train_df['label'])
    y_test = encoder.transform(test_df['label'])
    
 
    y_train = torch.LongTensor(y_train)
    y_test = torch.LongTensor(y_test)
    

    train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
    test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
    

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    # Initialize model
    model = SimpleCNN(num_classes=len(encoder.classes_))
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
   
    for epoch in range(epochs):
        model.train()
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    
   
    model.eval()
    with torch.no_grad():
        test_outputs = model(X_test)
        _, predicted = torch.max(test_outputs, 1)
        accuracy = (predicted == y_test).float().mean()
        
    print(f"\nResults [LR={lr}, BS={32}, Epochs={epochs}]:")
    print(f"Test Accuracy: {accuracy.item():.4f}")


   
    y_true = y_test.numpy()
    y_pred = predicted.numpy()
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=encoder.classes_))
    
   
    test_df['Predicted Label'] = encoder.inverse_transform(predicted.numpy())
    test_df.to_csv("test_predictions_cnn.csv", index=False)
    

In [19]:
run_CNN(0.01, 10)


Results [LR=0.01, BS=32, Epochs=10]:
Test Accuracy: 0.8950

Classification Report:
                precision    recall  f1-score   support

 museum-indoor       0.91      0.88      0.89       100
museum-outdoor       0.88      0.91      0.90       100

      accuracy                           0.90       200
     macro avg       0.90      0.90      0.89       200
  weighted avg       0.90      0.90      0.89       200



In [20]:
run_CNN(0.01, 20)


Results [LR=0.01, BS=32, Epochs=20]:
Test Accuracy: 0.8800

Classification Report:
                precision    recall  f1-score   support

 museum-indoor       0.85      0.92      0.88       100
museum-outdoor       0.91      0.84      0.88       100

      accuracy                           0.88       200
     macro avg       0.88      0.88      0.88       200
  weighted avg       0.88      0.88      0.88       200



In [21]:
run_CNN(0.001, 10)


Results [LR=0.001, BS=32, Epochs=10]:
Test Accuracy: 0.9050

Classification Report:
                precision    recall  f1-score   support

 museum-indoor       0.91      0.90      0.90       100
museum-outdoor       0.90      0.91      0.91       100

      accuracy                           0.91       200
     macro avg       0.91      0.91      0.90       200
  weighted avg       0.91      0.91      0.90       200



In [22]:
run_CNN(0.001, 20)


Results [LR=0.001, BS=32, Epochs=20]:
Test Accuracy: 0.9000

Classification Report:
                precision    recall  f1-score   support

 museum-indoor       0.92      0.88      0.90       100
museum-outdoor       0.88      0.92      0.90       100

      accuracy                           0.90       200
     macro avg       0.90      0.90      0.90       200
  weighted avg       0.90      0.90      0.90       200



In [23]:
run_CNN(0.0001, 10)


Results [LR=0.0001, BS=32, Epochs=10]:
Test Accuracy: 0.9150

Classification Report:
                precision    recall  f1-score   support

 museum-indoor       0.90      0.94      0.92       100
museum-outdoor       0.94      0.89      0.91       100

      accuracy                           0.92       200
     macro avg       0.92      0.92      0.91       200
  weighted avg       0.92      0.92      0.91       200



In [26]:
run_CNN(0.0001, 20)


Results [LR=0.0001, BS=32, Epochs=20]:
Test Accuracy: 0.9300

Classification Report:
                precision    recall  f1-score   support

 museum-indoor       0.91      0.95      0.93       100
museum-outdoor       0.95      0.91      0.93       100

      accuracy                           0.93       200
     macro avg       0.93      0.93      0.93       200
  weighted avg       0.93      0.93      0.93       200

